# Exploration of JSONs 
Small random batch of 100 datasets

#### Use [Nominatim]() for mapping of placenames/streets to coordinates 


In [44]:
from json_to_df import extract_data

# Extract data from json files
data_nogeo = extract_data(prefix = 'nogeo')
df_nogeo = pd.DataFrame(data_nogeo)

In [45]:
df_nogeo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   id                       110 non-null    object
 1   title                    110 non-null    object
 2   dsDescriptionValues      110 non-null    object
 3   dansSpatialCoverageText  110 non-null    object
dtypes: object(4)
memory usage: 3.6+ KB


In [47]:
df_nogeo.head()

,id,title,dsDescriptionValues,dansSpatialCoverageText
0,10.17026/dans-zh2-6ube,PAN-00132785 - seal with connecting strip and pin,<p>This find is registered at Portable Antiqui...,"[Amsterdam, NLD]"
1,10.17026/dans-286-dy7s,"Enschede (OV) - Wesselerbrink, deelgebied 5",<p>onderzoeksrapport</p>,"[Nederland, Het Leunenberg, Wesselerbrinklaan,..."
2,10.17026/AR/WIPZRT,"PAN-00105621 - coin/coin-related, provincie, duit",<p>This find is registered at Portable Antiqui...,"[Katwijk, NLD]"
3,10.17026/dans-zdc-usah,PAN-00036502 - button with solid convex head a...,<p>This find is registered at Portable Antiqui...,"[Goes, NLD]"
4,10.17026/dans-xup-tcgu,"PAN-00005610 - coin/coin-related, (maker : Han...",<p>This find is registered at Portable Antiqui...,"[Amsterdam, NLD]"


In [48]:
df.dsDescriptionValues = df.dsDescriptionValues.str.replace('<p>', '')
df.dsDescriptionValues = df.dsDescriptionValues.str.replace('</p>', '')

In [27]:
df

,id,title,dsDescriptionValues,dansSpatialCoverageText
0,10.17026/dans-z2t-fr88,Beerseweg 6 Oirschot,KSP Archeologie heeft een archeologisch bureau...,"[Oirschot, Noord-Brabant]"
1,10.17026/dans-xx7-w66k,NABO - locatie Beesel,Om het aantal incidenten op overwegen te vermi...,"[Beesel, Gemeente Beesel, Limburg]"
2,10.17026/dans-xdh-uzt5,"Bloemendaal, Brederodelaan 34 Gemeente Bloemen...",In februari 2015 is een archeologisch vooronde...,"[Brederodelaan 34, Bloemendaal, Gemeente Bloem..."
3,10.17026/dans-zeu-pe5h,"Locatie 'Dr. van Noortstraat 6' te Lienden, ge...",onderzoeksrapport,"[Dr. van Noortstraat, Nederland, Buren, Liende..."
4,10.17026/dans-z3c-p6t9,"bbbley 1, 2, 3 en 4 (Leyweg)",4 projecten samengevoegd: Leyweg 1412-1482 bor...,"[Den Haag, 's-Gravenhage, Zuid-Holland, Leyweg..."
...,...,...,...,...
245,10.17026/dans-24d-su5u,Bureauonderzoek en Inventariserend Veldonderzo...,Met dit onderzoek is de archeologische verwach...,"[Kampsestraat, Angeren, Gemeente Lingewaard, P..."
246,10.17026/AR/THR7DS,"Plangebied Resort Haamstede, gemeente Schouwen...",\nIn opdracht van CV Resort Haamstede UA heeft...,"[Daleboutsweg 4, Burgh, Gemeente Schouwen-Duiv..."
247,10.17026/dans-zfj-v2ak,"Stadsrandzone Zuid te Enschede, Deelgebied 2 en 3",Synthegra heeft in opdracht van de Gemeente En...,"[34F, Stadsrandzone Zuid, Enschede, Overijssel]"
248,10.17026/dans-zk6-dzdy,Een inventariserend archeologisch veldonderzoe...,Op 31 augustus 2004 is aan de Oosterseveldweg ...,[Weststellingwerf; Oosterstreek; Oosterseveldw...


In [10]:
df.to_csv('../data/missing_geo_test.csv', index=False)

In [11]:
geo_text = df.dansSpatialCoverageText.tolist()

In [16]:
geo_text[:10]

[['Amsterdam', 'NLD'],
 ['Nederland',
  'Het Leunenberg',
  'Wesselerbrinklaan',
  'Wesselerbrink',
  'Enschede',
  'Overijssel',
  'e-ne (MARC21)'],
 ['Katwijk', 'NLD'],
 ['Goes', 'NLD'],
 ['Amsterdam', 'NLD'],
 ['Zaltbommel', 'NLD'],
 ['Vught', 'NLD'],
 ['Maasdriel', 'NLD'],
 ['Echt-Susteren', 'NLD'],
 ['Utrecht', 'NLD']]

In [21]:
import time
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="DANS-geomapping")

for text in geo_text[:100]:
    for entry in text: 
        print(entry)
        location = geolocator.geocode(entry)
        if location:
            print(location.latitude, location.longitude)
            print()
            time.sleep(1)
        else: 
            print("NOT FOUND: ", entry)
            print()


Amsterdam
52.3730796 4.8924534

NLD
27.443653849999997 -99.56786380287167

Nederland
52.2434979 5.6343227

Het Leunenberg
52.1927163 6.8840567

Wesselerbrinklaan
52.1968624 6.8785649

Wesselerbrink
52.191438649999995 6.875410720999296

Enschede
52.2209855 6.8940537

Overijssel
52.4254143 6.4610611

e-ne (MARC21)
NOT FOUND:  e-ne (MARC21)

Katwijk
52.18985485 4.414558264500643

NLD
27.443653849999997 -99.56786380287167

Goes
51.5031033 3.8897596

NLD
27.443653849999997 -99.56786380287167

Amsterdam
52.3730796 4.8924534

NLD
27.443653849999997 -99.56786380287167

Zaltbommel
51.782164 5.189826737158967

NLD
27.443653849999997 -99.56786380287167

Vught
51.6511806 5.23680014410728

NLD
27.443653849999997 -99.56786380287167

Maasdriel
51.7853378 5.316792505584603

NLD
27.443653849999997 -99.56786380287167

Echt-Susteren
51.082135199999996 5.899125547533698

NLD
27.443653849999997 -99.56786380287167

Utrecht
52.0907006 5.1215634

NLD
27.443653849999997 -99.56786380287167

Deventer
52.26948965

### Evaluation 

In [1]:
# Grab 250 random datapoints that have geospatial data 
import pandas as pd
df = pd.read_csv('../data/archaeology_metadata.csv')
df_geo = df[df['dansSpatialBoxNorth'].notnull() | df['dansSpatialPointX'].notnull()]
df_geo = df_geo.sample(250)

In [2]:
df_geo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 250 entries, 122292 to 147710
Data columns (total 13 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   dsPersistentId                                        250 non-null    object 
 1   publicationStatus                                     250 non-null    object 
 2   title                                                 250 non-null    object 
 3   dsDescriptionValue                                    250 non-null    object 
 4   dansSpatialPointX                                     237 non-null    object 
 5   dansSpatialPointY                                     237 non-null    object 
 6   dansSpatialPointScheme                                237 non-null    object 
 7   dansSpatialBoxNorth                                   15 non-null     object 
 8   dansSpatialBoxEast                                    15 

In [3]:
geo_dois = df_geo.dsPersistentId.tolist()

In [12]:
# from doi_to_json import get_json

# for doi in geo_dois:
#     get_json(doi)

https://archaeology.datastations.nl/api/datasets/export?exporter=OAI_ORE&persistentId=doi:10.17026/dans-xqk-7yft
JSON data has been saved to '../jsons/geo_doi:10.17026%dans-xqk-7yft.json'.
https://archaeology.datastations.nl/api/datasets/export?exporter=OAI_ORE&persistentId=doi:10.17026/AR/OMPHMY
JSON data has been saved to '../jsons/geo_doi:10.17026%AR%OMPHMY.json'.
https://archaeology.datastations.nl/api/datasets/export?exporter=OAI_ORE&persistentId=doi:10.17026/dans-z9p-czud
JSON data has been saved to '../jsons/geo_doi:10.17026%dans-z9p-czud.json'.
https://archaeology.datastations.nl/api/datasets/export?exporter=OAI_ORE&persistentId=doi:10.17026/dans-2ak-d7bh
JSON data has been saved to '../jsons/geo_doi:10.17026%dans-2ak-d7bh.json'.
https://archaeology.datastations.nl/api/datasets/export?exporter=OAI_ORE&persistentId=doi:10.17026/dans-xdd-62cg
JSON data has been saved to '../jsons/geo_doi:10.17026%dans-xdd-62cg.json'.
https://archaeology.datastations.nl/api/datasets/export?exporte

In [1]:
from json_to_df import extract_data
import pandas as pd
data = extract_data()
df = pd.DataFrame(data)

In [3]:
df

,id,title,dsDescriptionValues,dansSpatialCoverageText,dansSpatialPoints,dansSpatialSchemes
0,10.17026/dans-z2t-fr88,Beerseweg 6 Oirschot,KSP Archeologie heeft een archeologisch bureau...,"[Oirschot, Noord-Brabant]","148651, 389591",RD (in m.)
1,10.17026/dans-xx7-w66k,NABO - locatie Beesel,Om het aantal incidenten op overwegen te vermi...,"[Beesel, Gemeente Beesel, Limburg]","202395, 364385",RD (in m.)
2,10.17026/dans-xdh-uzt5,"Bloemendaal, Brederodelaan 34 Gemeente Bloemen...",In februari 2015 is een archeologisch vooronde...,"[Brederodelaan 34, Bloemendaal, Gemeente Bloem...","102990, 492811",RD (in m.)
3,10.17026/dans-zeu-pe5h,"Locatie 'Dr. van Noortstraat 6' te Lienden, ge...",onderzoeksrapport,"[Dr. van Noortstraat, Nederland, Buren, Liende...","164000, 439847",RD (in m.)
4,10.17026/dans-z3c-p6t9,"bbbley 1, 2, 3 en 4 (Leyweg)",4 projecten samengevoegd: Leyweg 1412-1482 bor...,"[Den Haag, 's-Gravenhage, Zuid-Holland, Leyweg...","78919, 451961",RD (in m.)
...,...,...,...,...,...,...
245,10.17026/dans-24d-su5u,Bureauonderzoek en Inventariserend Veldonderzo...,Met dit onderzoek is de archeologische verwach...,"[Kampsestraat, Angeren, Gemeente Lingewaard, P...","194286, 436293",RD (in m.)
246,10.17026/AR/THR7DS,"Plangebied Resort Haamstede, gemeente Schouwen...",\nIn opdracht van CV Resort Haamstede UA heeft...,"[Daleboutsweg 4, Burgh, Gemeente Schouwen-Duiv...","40097, 412089",RD (in m.)
247,10.17026/dans-zfj-v2ak,"Stadsrandzone Zuid te Enschede, Deelgebied 2 en 3",Synthegra heeft in opdracht van de Gemeente En...,"[34F, Stadsrandzone Zuid, Enschede, Overijssel]","254892, 467617",RD (in m.)
248,10.17026/dans-zk6-dzdy,Een inventariserend archeologisch veldonderzoe...,Op 31 augustus 2004 is aan de Oosterseveldweg ...,[Weststellingwerf; Oosterstreek; Oosterseveldw...,"207900, 544400",RD (in m.)


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   id                       250 non-null    object
 1   title                    250 non-null    object
 2   dsDescriptionValues      250 non-null    object
 3   dansSpatialCoverageText  250 non-null    object
 4   dansSpatialPoints        250 non-null    object
 5   dansSpatialSchemes       237 non-null    object
dtypes: object(6)
memory usage: 11.8+ KB
